In [5]:
import tensorflow as tf 
from tensorflow import keras
from keras.models import load_model 
import cv2
import numpy as np
from gtts import gTTS
import os
from IPython.display import Audio
import smtplib
from email.message import EmailMessage


In [6]:
def email_alert(subject, body, to):
  msg = EmailMessage()
  msg.set_content(body)
  msg['subject'] = subject
  msg['to'] = to

  user = "3idiotsminiproject@gmail.com"
  msg['from'] = user
  password = "udjljundnfmglnxz"

  server = smtplib.SMTP("smtp.gmail.com", 587)
  server.starttls()
  server.login(user, password)
  server.send_message(msg)
  server.quit()


In [7]:
model = load_model('C:\\Users\\ADMIN\\Desktop\\Miniproject\\Face Mask Detection\\My_model')

face_clsfr = cv2.CascadeClassifier('C:\\Users\\ADMIN\\Desktop\\Miniproject\\Face Mask Detection\\haarcascade_frontalface_default.xml')

source = cv2.VideoCapture(0)

labels_dict={0:'NO MASK',1:'MASK'}
color_dict={0:(0,0,255),1:(0,255,0)}


In [8]:
while(True):

    #ret,img=cv2.imread(source)
    ret, img = source.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  


    for x,y,w,h in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

        if (label == 0):
           email_alert("SYSTEM ALERT!!!", "One Visitor violated Face Mask Policy. A Person has been detected without a mask","vritikapathare11@gmail.com")
           audio_en = gTTS('Please wear your mask correctly', lang='en')
           audio_hi = gTTS('कृपया अपना मास्क सही ढंग से पहनें    ', lang='hi')
           audio_mr = gTTS(' कृपया आपला मुखवटा योग्य प्रकारे घाला ', lang='mr')
           with open('alert.mp3', 'wb') as f:
              audio_en.write_to_fp(f)
              audio_hi.write_to_fp(f)
              audio_mr.write_to_fp(f)

           os.system("start alert.mp3")

           Audio("/content/alert.mp3", autoplay=True,rate=50)

    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)

    if(key==27):
        break
    
cv2.destroyAllWindows()
source.release()


ValueError: could not convert string to float: '/content/alert.mp3'